In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.iloc[:1170]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
df.set_index('Date', inplace=True)
df =df.apply(pd.to_numeric, errors='coerce')
df

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-3.77,-4.31,-2.37,1.95,-1.68,0.14,-0.40,-0.22,-5.29,-3.35
2023-09-01,-4.57,-2.58,-7.08,3.17,-5.98,-3.22,-5.68,-4.71,-5.04,-3.41
2023-10-01,-3.53,-17.88,-2.76,-6.24,-1.78,-0.18,0.47,-4.58,1.12,-2.53


In [12]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [13]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [ ]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()


rolling_pf_VAR(df, 60)

In [ ]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [18]:
def rolling_pf_inverse_volatility(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        sigma = window.cov()
        volatilities = np.sqrt(np.diag(sigma))  # Calculate volatilities
        
        def inverse_volatility(w):
            w = np.array(w)
            inv_vol = 1 / volatilities
            weighted_inv_vol = np.dot(w, inv_vol)
            return -weighted_inv_vol  # Maximize instead of minimize

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(inverse_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,110035296.850709,-65024584.872032,-29039688.010093,-16108529.769862,-70006560.804227,54672812.379444,-11174790.098057,44984964.064303,-46659094.510577,28320175.770393
1931-08-01,132354774.42208,-73706727.356174,-36946077.733379,-18453408.586145,-85766805.837466,64715863.057613,-12221973.358725,56013862.563926,-55698581.285822,29709075.114093
1931-09-01,403542488.302431,-220039160.769956,-110189158.575027,-64242699.290581,-258731004.161076,195198135.296842,-37206030.15057,169619913.38798,-167909879.786416,89957396.746381
1931-10-01,552857342069540.75,-311957886753937.6875,1086023613483.305908,10307071244.173752,-363022384216250.5,358474079748062.125,-683037795327.804321,-448008337455.073242,-236585685693334.375,269252355420.005615
1931-11-01,276889028329626.4375,-199077516024078.5625,4886423050056.070312,-2459898050632.966309,-226379244807081.625,177433108326755.5625,-5219024392311.845703,13389877871066.466797,-112673751269519.75,73210996507444.109375
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,80545034.502373,-127402698.54386,807173.845437,-102840645.554565,-11784960.251227,16901677.923196,11130194.902234,71907468.750469,67481458.965185,-6744703.539241
2023-09-01,65653373.276797,-106370498.295423,398548.76194,-85604261.861046,-8973679.024069,14600912.885882,11731260.721305,61407024.262168,53266816.032377,-6109495.759931
2023-10-01,83976193.764194,-136402546.698789,-829522.721068,-109444238.131355,-12168337.658669,20411949.927033,14468986.218306,78643763.170114,67930245.114105,-6586491.983871


In [14]:
#def rolling_pf_SameWeights(returns, window_size):

SyntaxError: incomplete input (3184422267.py, line 1)

In [ ]:
#def rolling_pf_MarketCap(returns, window_size):

In [16]:
def rolling_pf_minVAR(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size+1:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def min_volatility(w):
            w = np.array(w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            return V

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(min_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

rolling_pf_minVAR(df, window_size=60)


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,1.339827,-0.050735,-0.589697,0.171479,-0.107089,0.545689,0.001326,0.021876,-0.169376,-0.163299
1931-08-01,1.361932,-0.055047,-0.580431,0.171693,-0.089189,0.534139,-0.021838,0.023656,-0.196151,-0.148765
1931-09-01,1.353899,-0.047487,-0.585113,0.159202,-0.101621,0.529445,-0.019857,0.020704,-0.185995,-0.123178
1931-10-01,1.464908,-0.060773,-0.651862,0.137738,-0.073071,0.698601,0.007225,-0.131194,-0.302187,-0.089384
1931-11-01,1.480206,-0.089405,-0.613262,0.109025,-0.074769,0.680858,0.000105,-0.129991,-0.312133,-0.050634
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.629265,-0.03469,-0.561226,-0.103075,-0.024298,0.130757,0.147244,0.439241,0.272634,0.104149
2023-09-01,0.617147,-0.039232,-0.552363,-0.096857,-0.035298,0.151181,0.176889,0.447115,0.255834,0.075584
2023-10-01,0.642164,-0.033212,-0.631327,-0.093608,-0.050222,0.173209,0.176049,0.442272,0.265334,0.109342
